<a href="https://colab.research.google.com/github/jkcrosby3/DeerDetection/blob/main/DeerDetection_TensorFlow_with_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow with GPU

This notebook provides an introduction to computing on a [GPU](https://cloud.google.com/gpu) in Colab. In this notebook you will connect to a GPU, and then run some basic TensorFlow operations on both the CPU and a GPU, observing the speedup provided by using the GPU.


## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [2]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
6.081034302999996
GPU (s):
0.1333041769999994
GPU speedup over CPU: 45x


In [14]:
# Construct a tf.data.Dataset
ds = tfds.load('mnist', split='train', shuffle_files=True)
# Build your input pipeline
ds = ds.shuffle(1024).batch(32).prefetch(tf.data.AUTOTUNE)
for example in ds.take(1):
  image, label = example['image'], example['label']



In [23]:
#! pip install fiftyone
import textwrap
import fiftyone as fo
import fiftyone.zoo as foz
available_datasets = foz.list_zoo_datasets()
print(available_datasets)

['activitynet-100', 'activitynet-200', 'bdd100k', 'caltech101', 'caltech256', 'cifar10', 'cifar100', 'cityscapes', 'coco-2014', 'coco-2017', 'fashion-mnist', 'fiw', 'hmdb51', 'imagenet-2012', 'imagenet-sample', 'kinetics-400', 'kinetics-600', 'kinetics-700', 'kinetics-700-2020', 'kitti', 'kitti-multiview', 'lfw', 'mnist', 'open-images-v6', 'open-images-v7', 'places', 'quickstart', 'quickstart-3d', 'quickstart-geo', 'quickstart-groups', 'quickstart-video', 'sama-coco', 'ucf101', 'voc-2007', 'voc-2012']


In [ ]:
oi7dataset = foz.load_zoo_dataset('open-images-v7' )


cifar10_dataset = foz.get_zoo_dataset("cifar10")

#dataset = foz.load_zoo_dataset("cifar10",split="test",dataset_name="cifar10-test-sample",shuffle=True,max_samples=10,)

print("***** Dataset description *****")
print(textwrap.dedent("    " + oi7dataset.__doc__))

print("***** Tags *****")
print("%s\n" % ", ".join(oi7dataset.tags))

print("***** Supported splits *****")
print("%s\n" % ", ".join(oi7dataset.supported_splits))


INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/root/fiftyone/open-images-v7/train' if necessary


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable-with-rotation.csv' to '/root/fiftyone/open-images-v7/train/metadata/image_ids.csv'


 100% |██████|    4.8Gb/4.8Gb [28.2s elapsed, 0s remaining, 180.6Mb/s]      


INFO:eta.core.utils: 100% |██████|    4.8Gb/4.8Gb [28.2s elapsed, 0s remaining, 180.6Mb/s]      


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv' to '/root/fiftyone/open-images-v7/train/metadata/classes.csv'


...

INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/v5/train-masks/train-masks-6.zip' to '/root/fiftyone/open-images-v7/train/labels/masks/6.zip'


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/v5/train-masks/train-masks-c.zip' to '/root/fiftyone/open-images-v7/train/labels/masks/C.zip'


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/v5/train-masks/train-masks-1.zip' to '/root/fiftyone/open-images-v7/train/labels/masks/1.zip'


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/v5/train-masks/train-masks-7.zip' to '/root/fiftyone/open-images-v7/train/labels/masks/7.zip'


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/v5/train-masks/train-masks-5.zip' to '/root/fiftyone/open-images-v7/train/labels/masks/5.zip'


In [ ]:

cifar10_dataset = foz.get_zoo_dataset("cifar10")

#dataset = foz.load_zoo_dataset("cifar10",split="test",dataset_name="cifar10-test-sample",shuffle=True,max_samples=10,)

print("***** Dataset description *****")
print(textwrap.dedent("    " + oifar10_dataset.__doc__))

print("***** Tags *****")
print("%s\n" % ", ".join(oifar10_dataset.tags))

print("***** Supported splits *****")
print("%s\n" % ", ".join(oifar10_dataset.supported_splits))

# get
determine count of all classes of images
class 4 is deer (label y=4)
train on deer images
validate
test on everything
test

transfer
take pre-trained model
cut off last layer and replace with new model and run it again
